# TMDB

In [1]:
import json
with open(r"C:\Users\nbeac\.secret\tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [3]:
import os, time
import pandas as pd

In [4]:
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas_cleaned.csv.gz',
 'title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [5]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("Data/title_basics_cleaned.csv.gz")
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935 entries, 0 to 2934
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          2935 non-null   object 
 1   titleType       2935 non-null   object 
 2   primaryTitle    2935 non-null   object 
 3   originalTitle   2935 non-null   object 
 4   isAdult         2935 non-null   int64  
 5   startYear       2935 non-null   float64
 6   runtimeMinutes  2935 non-null   int64  
 7   genres          2935 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 183.6+ KB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000.0,86,"Musical,Romance"
2,tt0113092,movie,For the Cause,For the Cause,0,2000.0,100,"Action,Adventure,Drama"
3,tt0114447,movie,The Silent Force,The Silent Force,0,2001.0,90,Action
4,tt0115937,movie,Consequence,Consequence,0,2000.0,91,Drama


In [6]:
def write_json(new_data, filename, return_data=False): 
    """Appends the input json-compatible data to the json file.
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/
    Args:
        new_data (list or dict): json-compatible dictionary or list
        filename (str): json file to append data to

    Returns:
        return_data(bool): determines if combined data is returned (default =False) 
    """
    import json
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
                file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

    if return_data:
        return file_data

# API Extraction Function

In [7]:
# def get_movie_with_rating(movie_id):
#     #Get the movie object for the current id
#     movie = imdb.Movies(movie_id)
#     # save the .info .releases dictionaries
#     movie_info = movie.info()
#     releases = movie.releases()
#     # Loop through countries in releases
#     for c in releases['countries']:
#         if c['iso_3166_1'] =='US':
#             movie_info['certification'] = c['certification']
#     return movie_info

In [8]:
def get_movie_with_rating(movie_id):
    """Retrieve the dictionary of movie data using tmbdbsimple, including MPAA rating.
    Args:
        movie_id (str): movie-to-retreive's id (from IMDB's tconst column)
        
    Returns:
        dict: dictionary of movie.info() + the certification from movie.releases"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

# Before the Loops

In [9]:
YEARS_TO_GET = list(range(2000,2002))
YEARS_TO_GET

[2000, 2001]

# OUTER Loop

## Setting up a Progress Bar

Select a JSON_FILE filename to save the results in progress.
* Check if the file exists.
    * if no:
        * Create the empy JSON_FILE with "with open" that just contains the key "imdb_id"
            
    * if yes:
        * Do nothing!

The code below will identify the folder in the FOLDER we just define above and will name the file based on the current year

In [10]:
# OUTER LOOP
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    basics = pd.read_csv("Data/title_basics_cleaned.csv.gz")
    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue


    
    final_year_df = pd.read_json(JSON_FILE)
    
    csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)
    

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1409 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1526 [00:00<?, ?it/s]

In [11]:
# basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000.0,86,"Musical,Romance"
2,tt0113092,movie,For the Cause,For the Cause,0,2000.0,100,"Action,Adventure,Drama"
3,tt0114447,movie,The Silent Force,The Silent Force,0,2001.0,90,Action
4,tt0115937,movie,Consequence,Consequence,0,2000.0,91,Drama


In [12]:
tmdb_results_2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
tmdb_results_2000.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.000,0.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.107,1926.0,PG


In [13]:
tmdb_results_2001 = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\final_tmdb_data_2001.csv.gz")
tmdb_results_2001.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/ab5yL8zgRotrICzGbEl10z24N71.jpg,NaN,48000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,If they lived in the same century they'd be pe...,Kate & Leopold,0.0,6.316,1132.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,NaN
3,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.600,117.0,PG-13
4,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,NaN,1000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.100,28.0,R
